In [1]:
#Trying to fit an NN with strains and accumulation
#Trying for only 1 strain component
#If this doesn't work , i can peacefully follow that guy.

import tensorflow as tf
import numpy as np
def accumulation_generator(strains):
    """
    Generates accumulation vector of strains, used to capture path dependent behaviour.
    This is better option as it removes requirement of time(accumulation is basically essence of path dependency)
    and also is universal.
    
    
    
    Why I think adding time is not a good idea?
    if i train NN for strains at times 1,2.. but want to predict for strains at 0.1,0.2...? The predictions will be meaningless
    Using accumulations solves this problem since we deal only previously occured strains.
    """
    length = len(strains)
    print(length)
    accumulation = np.array([])
    for i in range(0,length):
        if i== 1 or i == 0:
            accumulation=np.append(accumulation,np.zeros(6))
            next
        else:
            temp = 0
            for j in range(2,i+1):
                temp=temp+np.abs(strains[j-1] - strains[j-2])
            accumulation=np.append(accumulation,np.array(temp))

    return np.array(accumulation)
        
node_list = []
for i in [0,36*3,36*5]:
    nodes_first_layer = [1,4,6,18,19,31,33,36]
    temp=[]
    temp = np.array(nodes_first_layer) + i
    print(temp)
    for j in temp:
        node_list.append(j)
for i in [36*1,36*4]:
    nodes_first_layer = [8,11,26,29]
    temp=[]
    temp = np.array(nodes_first_layer) + i
    print(temp)
    for j in temp:
        node_list.append(j)
for i in [36*2,36*3]:
    nodes_first_layer = [15,16,21,22]
    temp=[]
    temp = np.array(nodes_first_layer) + i
    print(temp)
    for j in temp:
        node_list.append(j)
additional_nodes=[16,16+(36*5),99,9+(36*3),20+(36*3),27+(36*3),23+(36*3),15+36,22+144]
additional_nodes.sort()

validational_nodes=[9+36,20+36,28+36,17+36,9+144,20+144,28+144,17+144,2,215,96]
validational_nodes.sort()

node_list.extend(additional_nodes)
node_list.extend(validational_nodes)
node_list.sort()

In [2]:
x=[]
y=[]


In [3]:

loading = np.genfromtxt('/home/vemu_sa/Downloads/DummyFE2Dataset_v01/Input/CompleteInput_60Nodes_6timesteps.txt',skip_header=1)
#accumulations = accumulation_generator(loading)
#x=np.array_split(np.append(loading,accumulations.reshape((10,6)),axis=1),10)
for j in x:
    x.append(j)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [4]:
for i in [99]:
    loading = np.loadtxt('/home/vemu_sa/Desktop/inputs_fe2/Loadingpaths_Nodes_sampling/loadingpath_node{}'.format(i))
    loading = np.delete(loading,0,1)
    accumulations = accumulation_generator(loading)
    x=np.array_split(np.append(loading,accumulations.reshape((10,6)),axis=1),10)
    for j in x:
        x_test.append(j)

10


In [5]:
for i in node_list:
    loading = np.loadtxt('/home/vemu_sa/Desktop/inputs_fe2/Loadingpaths_Nodes_sampling/strain_{}'.format(i))
    loading = loading[:,2]
    x = np.array_split(loading,10)
    for j in x:
        y_train.append(j.reshape((1,588)))
for i in [99]:
    loading = np.loadtxt('/home/vemu_sa/Desktop/inputs_fe2/Loadingpaths_Nodes_sampling/strain_{}'.format(i))
    loading = loading[:,2]
    x = np.array_split(loading,10)
    for j in x:
        y_test.append(j.reshape((1,588)))


In [6]:
len(x_test)==len(y_test)
len(x_train)==len(y_train)
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
model = Sequential()
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(588))

model.compile(loss='mse',optimizer='adam',metrics=['mse'])

In [ ]:


def fit_model(model):
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=200)
    history = model.fit(x_train,y_train,epochs=2000,callbacks=[early_stop],
                       validation_data=(x_test,y_test),batch_size=1)
    return history

In [ ]:
history = fit_model(model)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train loss','validation loss'], loc='upper right')

In [ ]:
nodes=np.arange(1,589,1)
plt.scatter(nodes,y_test[-1],label='Real')
y=model.predict(x_test[-1])
plt.scatter(nodes,y,label='Prediction')
plt.title('Strain component 3-normalized')
plt.legend()


In [ ]:
error = y_test[-1] - y

plt.violinplot([y.reshape((588,)),y_test[-1].reshape((588,))])

np.average(error)